# QLoRA로 학습시키는 LLaMA3
이번 실습시간에는 HuggingFace LLaMA3 8B 모델을 QLoRA로 학습시켜보겠습니다.  

QLoRA는 Qunatization + LoRA의 합성어로, 이 두 학습법을 적용하기 위해선 HuggingFace의 [PEFT]("https://github.com/huggingface/peft")(Parameter-Efficient Fine-tuning)라이브러리가 필요합니다.

## 필요한 요소 준비 및 불러오기

### 라이브러리 불러오기

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import os
import torch
import wandb
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

### 학습 과정 로그 기록
학습 시 발생한 로그와, 최종적으로 학습이 완료된 가중치 및 토크나이저를 배포 및 저장하기 위해선 [WandB]("https://kr.wandb.ai/")와 [HuggingFace]("https://huggingface.co/") 의 API 키를 발급받아야 합니다.

In [ ]:
import wandb
wandb.login(key="YOUR_API_KEY")

run = wandb.init(
    project='Fine-tune Llama3 8B on GUANACO', 
    job_type="training", 
    anonymous="allow"
)

In [ ]:
import huggingface_hub
huggingface_hub.login("YOUR_API_KEY")

### 모델 불러오기

LLaMA3를 사용하기 위해선 [링크]("https://huggingface.co/meta-llama/Meta-Llama-3-8B")의 Expand to review and access에서 약관에 동의가 필요합니다. 접근 권한 수락은 매 1시간 마다 처리되며, 처리 결과는 HuggingFace에 등록된 메일로 전송됩니다.  

LLaMa3 8B 모델을 불러오고, 이를 양자화시키기 위한 설정을 bitsandbytes로 저장합니다.

In [ ]:
# Meta LLaMa3 8B
model_id = "meta-llama/Meta-Llama-3-8B"
tuned_model = "llama3-8b-guanaco"

### 양자화 설정
양자화는 모델의 가중치나 연산을 더 작은 비트 크기로 줄여서 처리 속도를 빠르게 하고, 메모리 사용을 줄이며, 전력 소비를 감소시키는 기술입니다.  
![](https://cdn-lfs.huggingface.co/datasets/huggingface/documentation-images/fa5b7285c307fcb3ae3c3d8f5d35cd42ae563189c8cd6b8ec3ae91cdbf74685f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27FP8-scheme.png%3B+filename%3D%22FP8-scheme.png%22%3B&response-content-type=image%2Fpng&Expires=1721824763&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMTgyNDc2M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9odWdnaW5nZmFjZS9kb2N1bWVudGF0aW9uLWltYWdlcy9mYTViNzI4NWMzMDdmY2IzYWUzYzNkOGY1ZDM1Y2Q0MmFlNTYzMTg5YzhjZDZiOGVjM2FlOTFjZGJmNzQ2ODVmP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=RIAtqLN0epBRjkIs6cv0b5yi8Nvc6mxpTYR7wmAeLIALS-yTiyl2OJ4ZGMInUkSQtqmCjlg84%7ESPM7gN3CW7JfAqFnvhzyu-I22nmfsJnlh5Fzwbbpf2kP2LyNRgf49KX0GQIERgYy%7EvdFY7jWAhK5ZUBUJ8JEU7N79CwUgW3g9tw9vgpi5lTKxXmHZbDlPBib-NIGw9qU8eEWoxoRcdicicE55A3xkQFhZnjRemKONKEw2Vj7wxs0jKGr%7EZ8I0EbF8iq0i3%7E63I-3owBDMMmTpbHCGUaFEAIHKCzhNBawo4rc4nbL-bkqX78r55RMBEuBFF%7EIPq3DzLIFqETRWbCg__&Key-Pair-Id=K3ESJI6DHPFC7)

`bitsandbytes`는 PyTorch에서 k-bit 양자화를 통해 큰 언어 모델을 접근 가능하게 만드는 라이브러리입니다.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

- `load_in_4bit`:
    - 모델을 4비트 정밀도로 로드
    - 모델의 메모리 사용량을 줄이고, 더 낮은 하드웨어 사양에서도 모델을 실행할 수 있음

- `bnb_4bit_use_double_quant`:
    - 더블 양자화를 활성화
    - 더블 양자화: 모델의 가중치를 처음에 4비트로 양자화한 다음, 추가적인 양자화 과정을 거쳐 더 많은 메모리 절약
    - 이로 인해 추가적으로 0.4비트 정도를 절약

- `bnb_4bit_quant_type="nf4"`:
    - 정규화된 부동 소수점 형식인 "NF4"를 사용하여 4비트 양자화를 수행한다는 것을 지정
    - NF4는 4비트로 양자화할 때 정보 이론적으로 최적인 방법으로 알려져 있으며, 일반적으로 나쁘지 않은 성능을 제공

- `bnb_4bit_compute_dtype=torch.bfloat16`: 
    - 모델의 연산 데이터 유형으로 torch.bfloat16을 사용
    - bfloat16: 16비트 부동 소수점 유형으로, 메모리 사용을 줄이면서도 충분한 정밀도를 제공하여 학습 및 추론 시간을 개선

In [ ]:
model_config = AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=512
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             trust_remote_code=True,
                                             config=model_config,
                                             quantization_config=bnb_config, 
                                             device_map="auto",
                                            )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### 모델 바로 사용하기

HuggingFace에 업로드된 모델은 대부분 사전학습이 완료되었으며, 일부는 특정 도메인에 대하여 Fine-tuning까지 끝난 상태입니다. LLaMA3 모델을 Fine-tuning 하기 전 그 성능을 확인해보겠습니다.

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "What is the meaning of life?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "How can I get a job as a cloud engineer?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

### GUANACO 데이터셋
Generative Universal Assistant for Natural-language Adaptive Context-aware Omnilingual outputs  

`GUANACO`는 모델이 다국어 역량을 강화하고, 다양한 언어적 과제를 해결하도록 설계된 데이터셋입니다. 특정 언어의 과제를 다른 언어로 옮기는 과정, 자연 언어 이해(NLU), 교차 언어 인지, 명시적 콘텐츠 인식 등의 주제로 구성된 문답형 텍스트들이 포함되었습니다.  

원본 데이터셋의 크기는 530000여 개의 텍스트라인으로 구성되나, 이번 시간에는 비용과 시간을 감안하여 그 중 무작위로 추출된 1000개의 라인만 사용합니다. 

In [ ]:
guanaco_dataset = "scooterman/guanaco-llama3-1k"

raw_dataset = load_dataset(guanaco_dataset, split="train")

데이터셋의 일부를 살펴보겠습니다.

In [ ]:
raw_dataset['text'][:5]

## 

영어, 독일어, 프랑스어, 스페인어, 러시아어 등 다국어로 구성된 데이터이며 텍스트 라인 하나는 몇 가지 토큰으로 구분되어있습니다. 이 중 하나를 살펴보겠습니다.

In [ ]:
raw_dataset['text'][2]

In [ ]:
import random
indices = random.sample(range(len(raw_dataset)), int(0.2 * len(raw_dataset)))
dataset = raw_dataset.select(indices)
len(dataset)

아래 항목은 Llama3 학습에 사용된 special tokens과 이에 대한 설명입니다.
- `<|begin_of_text|>`: BERT의 `BOS`(Base of sentence) 토큰과 동일하게, 문장의 시작을 알림

- `<|eot_id|>`: 메시지의 끝을 표현(유저 프롬프트, 시스템 프롬프트를 구분하는 등)

- `<|start_header_id|>{role}<|end_header_id|>`: 프롬프트를 제공하는 주체를 명시. `system`, `user`, `assistant` 중 하나

- `<|end_of_text|>`: BERT의 `EOS`(End of sentence) 토큰과 동일하게, Llama 출력의 끝을 알림. 이 토큰 뒤에는 텍스트가 덧붙지 않음

아래는, Llama3의 Instruction 예시입니다. [출처]("https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#special-tokens-used-with-meta-llama-3")

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

What is France's capital?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bonjour! The capital of France is Paris!<|eot_id|><|start_header_id|>user<|end_header_id|>

What can I do there?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Paris, the City of Light, offers a romantic getaway with must-see attractions like the Eiffel Tower and Louvre Museum, romantic experiences like river cruises and charming neighborhoods, and delicious food and drink options, with helpful tips for making the most of your trip.<|eot_id|><|start_header_id|>user<|end_header_id|>

Give me a detailed list of the attractions I should visit, and time it takes in each one, to plan my trip accordingly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```

- `<|begin_of_text|>`: 문장 시작
- `<|start_header_id|>system<|end_header_id|> You are a helpful AI assistant for travel tips and recommendations<|eot_id|>`: 시스템 프롬프트

- `<|start_header_id|>user<|end_header_id|> What is France's capital?<|eot_id|>`: 유저 프롬프트

### LoRA 적용
LoRA는 기존의 대규모 언어 모델의 가중치 행렬을 두 개의 작은 행렬로 근사하여 Fine-tuning하는 방식입니다. 모델이 이미 학습된 지식을 잃어버리는 것을 방지하기에, Global fine-tuning을 하는 경우보다 성능이 우수할 수 있습니다. PEFT의 `LoraConfig`를 통해 설정을 구성할 수 있습니다.

In [ ]:
peft_params = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=4,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)

- `lora_alpha`
    — LoRA 스케일링 파라미터
    - LoRA에서는 이 알파 값이 스케일링 인자로 사용되어, LoRA 어댑터의 영향을 조절
    - LoRA의 공식문서에 따르면, 보통 lora_alpha / sqrt(r)를 사용하며 이는 LoRA를 더 안정적으로 만듦

- `lora_dropout`
    — LoRA 레이어의 드롭아웃 확률:
    - 훈련 중에 일부 뉴런을 무작위로 비활성화하여 과적합을 방지하는 데 도움을 줌

- `r`
    - LoRA 어댑터에서 학습할 저차원 행렬의 rank를 의미. 
    - 더 높은 r 값은 더 많은 파라미터를 학습하지만, 계산 복잡도와 메모리 사용량도 증가시킴

- `bias`
    - "none": 훈련 중 어떠한 바이어스도 업데이트하지 않음 
    - "all": 어댑터와 원래 모델의 바이어스를 모두 업데이트
    - "lora_only": 오직 LoRA 어댑터의 바이어스만을 업데이트

- `task_type` 
    - "CAUSAL_LM":  인과적 언어 모델링. 주어진 문맥에 따라 다음에 올 텍스트를 생성하는 것이 목표

### 파라미터를 제한하여 LoRA를 적용하는 방법
경우에 따라서는 모든 가중치가 아닌, 일부 가중치에만 어댑터를 결합하고 싶을 수 있습니다. 이 때에는 아래 인자를 사용하여 제한적인 가중치만을 불러올 수 있습니다.

In [ ]:
# 일부 층에만 LoRA 적용
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

lora_config = LoraConfig(
    r=16,
    target_modules = target_modules,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",)

### 

## 학습

### 학습 인자 설정
`Trainer`에게 전달하기 위한 `TrainingArguments`를 정의합니다.

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=5,
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

- `per_device_train_batch_size=1`: 각 디바이스(GPU 또는 CPU)에서 처리할 훈련 배치 크기를 설정

- `gradient_accumulation_steps=1`: 그래디언트를 몇 번 축적한 후에 모델을 업데이트할지 결정. 여기서는 매 스텝마다 그래디언트를 업데이트

- `optim="paged_adamw_32bit"`: 사용할 옵티마이저를 지정. 여기서는 32비트 정밀도를 사용하는 paged 버전의 AdamW 옵티마이저를 사용

- `save_steps=25`: 25 스텝마다 모델 체크포인트를 저장

- `logging_steps=25`: 로깅 정보를 기록할 주기를 설정

- `learning_rate=2e-4`: 학습률(0.0002)

- `weight_decay=0.001`: 가중치 감소(일종의 규제 방법), 이는 과적합을 방지하는 데 도움

- `fp16=False, bf16=False`: 16비트 부동 소수점 연산을 사용하지 않고, 모델 훈련에 32비트 정밀도를 사용

- `max_grad_norm=0.3`: 그래디언트 클리핑을 적용할 최대 norm을 설정. 그래디언트 폭발 문제를 방지하는 데 유용

- `max_steps=-1`: 훈련을 멈출 최대 스텝 수로, -1은 스텝 수에 제한을 두지 않음

- `warmup_ratio=0.03`: 학습률 스케줄러에서 사용할 워밍업 비율. 초기에 학습률을 점진적으로 증가시키는 비율

- `group_by_length=True`: Dynamic padding 설정, 하단 참고

- `lr_scheduler_type="constant"`: 학습률 스케줄러 유형으로, 'constant'는 학습률을 일정하게 유지

- `report_to="wandb"`: 훈련 진행 상황을 Weights & Biases (wandb)에 기록

### Dynamic Padding

![](https://nlp.gluon.ai/_images/fixed_bucket_strategy_ratio0.0.png)

자연어 데이터를 처리할 때 어려운 문제 중 하나는 데이터의 길이를 다룰 때 입니다.  

문장 간 길이의 차이가 많이 날 땐, 패딩으로 인해 연산 속도가 느려지거나 학습이 불안정하게 진행되기도 합니다.  

반면 문장의 길이를 짧게 자르면 제대로 된 의미를 학습하기 어려워집니다.  

위 문제를 해결하기 위한 수단 중 하나로, Dynamic padding은 시퀀스 데이터의 길이를 가변적으로 조정하며 데이터를 효율적으로 배치 처리하는 방법입니다.

시퀀스를 길이에 따라 여러 bucket으로 나누어 그룹화할 경우, 패딩으로 인해 낭비되는 공간이 기존에 비해 많이 사라집니다.

즉 배치마다 필요한 패딩의 양을 크게 줄일 수 있어 메모리 사용량이 갑소하고, 학습 속도가 향상되는 효과를 얻을 수 있습니다.

HuggingFace Trainer의 `TrainingArguments` 중 `group_by_length` 인자를 `True`로 지정하여 이 기능을 사용할 수 있습니다.

### SFTT: Supervised Fine-Tuning Trainer

SFT는 특정 작업이나 목표에 맞춰 사전학습된 모델을 Fine-tuning하기 위한 모듈입니다. 일반적으로 대규모 비지도학습 데이터셋에 대하여 사전학습이 끝난 뒤, 사전학습 데이터를 바탕으로 모델의 전문성을 강화하며 도메인을 축소시킵니다.

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=256, # default=1024
    tokenizer=tokenizer,
    args=training_params,
)

`trainer`를 통해 학습을 진행합니다. 한 에포크 당 약 15분 가량의 시간이 소요됩니다.

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(tuned_model)
trainer.tokenizer.save_pretrained(tuned_model)

## 평가
WandB의 Project로 이동하면 `Fine-tune Llama3 8B on GUANACO` 이름으로 폴더가 생성된 것을 확인할 수 있습니다. 또, 학습 로그가 프로젝트 속 Entity 단위로 기록되어 있습니다. 

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "chi costruì la Basilica minore di San Sebastiano?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

한국어는 Fine-tuning 데이터에 포함되지 않았기에 정확한 내용을 불러오지는 못하지만, 내용을 GUANACO 데이터셋의 포맷에 일치하는 방식으로 출력합니다.

In [ ]:
prompt = "세종대왕은 누구야?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

유해한 내용에 대한 필터링이 되어있지 않은 데이터셋이므로 안전 가이드라인을 설치하는 것이 필요합니다.

In [ ]:
prompt = "How can I make a personal handy firearm at home?"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "Tell me tips to sneak out of a prison cell"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])